This file is very similar to the "binary_90_minutes_LSTM.ipynb". However, the difference is that we will be keeping information about each team. Previously, we considered the events of the teams as a whole, in which we only cared about whether a given event happened in a given minute of a game regardless of what team committed it. Just like before, we will try to predict the binary outcomes of every minute of a game's second half; the only difference is that we will training and predicting binary outcomes for each team.

We do data cleaning along the way in which we generate 36 features that we save to a file, "rnn_data.csv". We then make sure that every game contains 90 minutes and write our adjustments to "rnn_data_cleaned.csv". We then use the latter file for our LSTM model.

In [0]:
import pandas as pd
import numpy as np

from tqdm import tqdm
from tqdm import tqdm_notebook

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
filename = "drive/Team Drives/Deep Learning Project/game_with_features.csv"
df = pd.read_csv(filename)
df.head(5)

,Unnamed: 0,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,...,Straight_Red_Card,Substitution,Free_Kick_Awarded,Off_Sides,Hand_Ball,Penalty_Awarded,Key_Pass,Failed_Through_Ball,Sending_Off,Own_Goal
0,0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,...,0,0,0,0,0,0,0,0,0,0
1,1,UFot0hit/,UFot0hit2,2,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,NaN,1,Borussia Dortmund,...,0,0,0,0,0,0,0,0,0,0
2,2,UFot0hit/,UFot0hit3,3,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,NaN,1,Borussia Dortmund,...,0,0,0,0,0,0,0,0,0,0
3,3,UFot0hit/,UFot0hit4,4,7,Foul by Sven Bender (Borussia Dortmund).,3,NaN,1,Borussia Dortmund,...,0,0,0,0,0,0,0,0,0,0
4,4,UFot0hit/,UFot0hit5,5,7,Gokhan Tore (Hamburg) wins a free kick in the ...,8,NaN,2,Hamburg SV,...,0,0,1,0,0,0,0,0,0,0


In [0]:
# create the game-based result dictionary since we need to match the result in the ANN.ipynb
import pandas as pd
from  collections import Counter
from copy import copy

df = pd.read_csv("drive/Team Drives/Deep Learning Project/events.csv")
df = df.ix[:,[0,8,9,16]]
game = df['id_odsp'].unique()
game_base = df.groupby('id_odsp')

game_dic = {}
for i in game:
    game_dic[i]=0
    
for each in game:
    this_game = game_base.get_group(each)
    this_game_goal = this_game.groupby('is_goal')


    if len(this_game['is_goal'].unique()) ==2:
        goal = this_game_goal.get_group(1)
        if len(goal['event_team'].unique())==2:
            a = dict(Counter(goal['event_team']))
            temp = copy(a)
            if list(temp.values())[0] > list(temp.values())[1]:
                a['win']=list(temp.keys())[0]
                a['loss']=list(temp.keys())[1]
                a['draw']=None
            else:
                if list(temp.values())[0] == list(temp.values())[1]:
                        a['win']=None
                        a['loss']=None
                        a['draw']=list(temp.keys())
                else:
                    a['win']=list(temp.keys())[1]
                    a['loss']=list(temp.keys())[0]
                    a['draw']=None
            game_dic[each]=a
        else:
            b = dict(Counter(goal['event_team']))
            temp = copy(b)
            other_team = list(this_game['event_team'].unique())
            for i in other_team:
                if list(temp.keys())[0] != i:
                    b[i]=0
                    opponent = i
            b['win'] = list(temp.keys())[0]
            b['loss'] = opponent
            b['draw'] = None
            game_dic[each]=b
    else:
        team = this_game['event_team'].unique()
        c = {}
        for i in team:

            c[i]=0
        temp = copy(c)
        c['win']=None
        c['loss']=None
        c['draw']=list(temp.keys())
        game_dic[each]=c

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [0]:
df.columns

Index(['id_odsp', 'event_team', 'opponent', 'is_goal'], dtype='object')

In [0]:
filename = "drive/Team Drives/Deep Learning Project/game_with_features.csv"
df = pd.read_csv(filename)
df1 = df[['id_odsp','time', 'event_team','is_goal', 'assist_method', 'fast_break', 'Shot_Attempted', 'Corner_Kick', 
          'Foul','First_Yellow_Card', 'Second_Yellow_Card', 'Straight_Red_Card', 'Substitution', 'Free_Kick_Awarded', 'Off_Sides', 'Hand_Ball', 'Penalty_Awarded', 'Key_Pass', 'Failed_Through_Ball', 'Sending_Off', 'Own_Goal',  ]]

is_goal0= []
assist_method0=[]
fast_break0 = []
Shot_Attempted0=[]
Corner_Kick0 = []
Foul0 = []
First_Yellow_Card0 = []
Second_Yellow_Card0=[]
Straight_Red_Card0 = []
Substitution0 = []
Free_Kick_Awarded0 = []
Off_Sides0 = []
Hand_Ball0 = []
Penalty_Awarded0 = []
Key_Pass0 = []
Failed_Through_Ball0=[]
Sending_Off0 = []
Own_Goal0 =[]

is_goal1= []
assist_method1=[]
fast_break1 = []
Shot_Attempted1=[]
Corner_Kick1 = []
Foul1 = []
First_Yellow_Card1 = []
Second_Yellow_Card1=[]
Straight_Red_Card1 = []
Substitution1 = []
Free_Kick_Awarded1 = []
Off_Sides1= []
Hand_Ball1 = []
Penalty_Awarded1 = []
Key_Pass1 = []
Failed_Through_Ball1=[]
Sending_Off1 = []
Own_Goal1 =[]
time = []
game_id = []
feature0 = [is_goal0, assist_method0, fast_break0, Shot_Attempted0,Corner_Kick0, Foul0,First_Yellow_Card0, Second_Yellow_Card0, 
            Straight_Red_Card0, Substitution0, Free_Kick_Awarded0, Off_Sides0, Hand_Ball0, Penalty_Awarded0, Key_Pass0, Failed_Through_Ball0, Sending_Off0, Own_Goal0]

feature1 = [is_goal1, assist_method1, fast_break1, Shot_Attempted1,Corner_Kick1, Foul1,First_Yellow_Card1, Second_Yellow_Card1, 
            Straight_Red_Card1, Substitution1, Free_Kick_Awarded1, Off_Sides1, Hand_Ball1, Penalty_Awarded1, Key_Pass1, Failed_Through_Ball1, Sending_Off1, Own_Goal1]
feature = feature0+feature1

In [0]:
game = list(df1['id_odsp'].unique())

game_group = df1.groupby('id_odsp')

for game_count in tqdm(game):
  temp1 = game_group.get_group(game_count)
  time_group = temp1.groupby('time')
  unique_time = temp1['time'].unique()
  for time_count in unique_time:
    time.append(time_count)
    temp2 = time_group.get_group(time_count)
    unique_team = list(temp2['event_team'].unique())
    game_id.append(game_count)
    if len(unique_team) == 2:
      for team_count in unique_team:
        temp3 = temp2[temp2['event_team']==team_count]
        summ = temp3.sum()
        del summ['id_odsp']
        del summ['event_team']
        del summ['time']
        summ = list(summ)
        if team_count == list(game_dic[game_count].keys())[0]:            
            for i in range(18):
                feature[i].append(summ[i])
        else:
            for i in range(18):
                feature[i+18].append(summ[i]) 
    if len(unique_team) == 1:
       for team_count in unique_team:
          temp3 = temp2[temp2['event_team']==team_count]
          summ = temp3.sum()
          del summ['id_odsp']
          del summ['event_team']
          del summ['time']
          summ = list(summ)
          if team_count == list(game_dic[game_count].keys())[0]:            
              for i in range(18):
                  feature[i].append(summ[i])
                  feature[i+18].append(0)
          else:
              for i in range(18):
                  feature[i+18].append(summ[i])
                  feature[i].append(0)
    if len(unique_team) == 0:
        for i in range(36):
          feature[i].append(0)

100%|██████████| 9074/9074 [25:21<00:00,  5.74it/s]


In [0]:
# generate the 36 features data
df_final = pd.DataFrame({'id_odsp': game_id, 'time':time,'is_goal0': is_goal0, 'assist_method0':assist_method0,'fast_break0': fast_break0, 'Shot_Attempted0': Shot_Attempted0,'Corner_Kick0': Corner_Kick0, 
                         'Foul0': Foul0,'First_Yellow_Card0': First_Yellow_Card0, 'Second_Yellow_Card0':Second_Yellow_Card0, 
            'Straight_Red_Card0':Straight_Red_Card0, 'Substitution0':Substitution0, 'Free_Kick_Awarded0': Free_Kick_Awarded0, 'Off_Sides0':Off_Sides0, 'Hand_Ball0':Hand_Ball0, 'Penalty_Awarded0':Penalty_Awarded0, 
                         'Key_Pass0':Key_Pass0, 'Failed_Through_Ball0': Failed_Through_Ball0, 'Sending_Off0':Sending_Off0, 'Own_Goal0':Own_Goal0,'is_goal1': is_goal1, 'assist_method1':assist_method1,'fast_break1': fast_break1, 
                         'Shot_Attempted1': Shot_Attempted1,'Corner_Kick1': Corner_Kick1, 
                         'Foul1': Foul1,'First_Yellow_Card1': First_Yellow_Card1, 'Second_Yellow_Card1':Second_Yellow_Card1, 
            'Straight_Red_Card1':Straight_Red_Card1, 'Substitution1':Substitution1, 'Free_Kick_Awarded1': Free_Kick_Awarded1, 'Off_Sides1':Off_Sides1, 'Hand_Ball1':Hand_Ball1, 'Penalty_Awarded1':Penalty_Awarded1, 
                         'Key_Pass1':Key_Pass1, 'Failed_Through_Ball1': Failed_Through_Ball1, 'Sending_Off1':Sending_Off1, 'Own_Goal1':Own_Goal1})
df_final.to_csv("drive/Team Drives/Deep Learning Project/ann/rnn_data.csv", index = False)

In [0]:
df_final.head(10)

,Corner_Kick0,Corner_Kick1,Failed_Through_Ball0,Failed_Through_Ball1,First_Yellow_Card0,First_Yellow_Card1,Foul0,Foul1,Free_Kick_Awarded0,Free_Kick_Awarded1,...,Substitution0,Substitution1,assist_method0,assist_method1,fast_break0,fast_break1,id_odsp,is_goal0,is_goal1,time
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,UFot0hit/,0,0,2
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,UFot0hit/,0,0,4
2,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,UFot0hit/,0,0,7
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,UFot0hit/,0,0,9
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,UFot0hit/,0,0,10
5,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,UFot0hit/,0,0,11
6,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,UFot0hit/,0,0,13
7,0,0,0,0,0,0,0,1,1,0,...,0,0,1,0,0,0,UFot0hit/,0,0,14
8,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,UFot0hit/,1,0,17
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,UFot0hit/,0,0,19


In [0]:
# clean process
filename = "drive/Team Drives/Deep Learning Project/ann/rnn_data.csv"
df = pd.read_csv(filename)
df = df.dropna()

#Make sure that each game has an event sequence for time 0 to 90.
iterables = [df['id_odsp'].unique(), list(range(0, 90))]
df = df.set_index(['id_odsp','time'])
df = df.reindex(index=pd.MultiIndex.from_product(iterables, names=['id_odsp', 'time']), fill_value= 0).reset_index()
df.drop(df[df.time > 89].index, inplace=True)  #Remove rows from a game that are over 90 minutes. We are not accounting for overtime.

df.to_csv("drive/Team Drives/Deep Learning Project/ann/rnn_data_cleaned", index = False)
#os.system('say "Hey, Armaan, your program has finished."')


In [0]:
df.head(10)

,id_odsp,time,Corner_Kick0,Corner_Kick1,Failed_Through_Ball0,Failed_Through_Ball1,First_Yellow_Card0,First_Yellow_Card1,Foul0,Foul1,...,Straight_Red_Card0,Straight_Red_Card1,Substitution0,Substitution1,assist_method0,assist_method1,fast_break0,fast_break1,is_goal0,is_goal1
0,UFot0hit/,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,UFot0hit/,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,UFot0hit/,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,UFot0hit/,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,UFot0hit/,4,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,UFot0hit/,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,UFot0hit/,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,UFot0hit/,7,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
8,UFot0hit/,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,UFot0hit/,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
training_set = df.as_matrix() # Using multiple predictors.

X = []
Y = []
    
current_game = ""

new_row_X = []
new_row_Y = []

for index, row in enumerate(training_set):
    if current_game == row[0]:
        if row[1] <= 44:
            #print("Row before:", row)
            row = np.delete(row, 0) #drop the game id from the row
            row = np.delete(row, 0) #drop the time from the row
            #print("Row after:", row)
            new_row_X.append(row)
        else:
            row = np.delete(row, 0)
            row = np.delete(row, 0)
            new_row_Y.append(row)
    else:
        if index != 0:
            X.append(new_row_X)
            Y.append(new_row_Y)
        new_row_X = []
        new_row_Y = []
        current_game = row[0]
        if row[1] <= 45:
            row = np.delete(row, 0)
            row = np.delete(row, 0)
            new_row_X.append(row)
        else:
            row = np.delete(row, 0)
            row = np.delete(row, 0)
            new_row_Y.append(row)

In [0]:
# Import Libraries and packages from Keras
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.layers import Dense, Activation, Flatten
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
# Initializing the RNN
lstm_model = Sequential()
X = np.array(X)
Y = np.array(Y)


#Adding the first LSTM layer and some Dropout regularization.
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(45, 36)))
lstm_model.add(Dropout(0.2))

#Adding a second LSTM layer and some Dropout regularization.
lstm_model.add(LSTM(units=50, return_sequences=True))
lstm_model.add(Dropout(0.2))

#Adding a third LSTM layer and some Dropout regularization.
lstm_model.add(LSTM(units=50, return_sequences=True))
lstm_model.add(Dropout(0.2))

#Adding a fourth LSTM layer and some Dropout regularization.
lstm_model.add(LSTM(units=50, return_sequences = True))
lstm_model.add(Dropout(0.2))

#Adding the output layer.
lstm_model.add(Dense(units = 36, activation = 'sigmoid'))

Using TensorFlow backend.


In [0]:
lstm_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 45, 50)            17400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 45, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 45, 50)            20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 45, 50)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 45, 50)            20200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 45, 50)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 45, 50)            20200     
__________

In [0]:
X.shape

(9073, 45, 36)

In [0]:
Y.shape

(9073, 45, 36)

In [0]:
# Compiling the RNN
lstm_model.compile(optimizer='adam', loss="binary_crossentropy", metrics = ['acc'])  # Can change loss to mean-squared-error if you require.
lstm_model.fit(X, Y, epochs = 2, batch_size = 32, validation_split = 0.33)

Train on 6078 samples, validate on 2995 samples
Epoch 1/2
6078/6078 [==============================] - 113s 19ms/step - loss: 0.2379 - acc: 0.9422 - val_loss: 0.1448 - val_acc: 0.9625
Epoch 2/2
6078/6078 [==============================] - 109s 18ms/step - loss: 0.1479 - acc: 0.9611 - val_loss: 0.1371 - val_acc: 0.9625


In [0]:
a = lstm_model.predict(X)

In [0]:
a = np.array([ np.mean(i, axis = 0) for i in a])

In [0]:
a.shape

(9073, 36)

In [0]:
np.savetxt('drive/Team Drives/Deep Learning Project/rnn_pred.txt',a)